In [23]:
import pandas as pd
import numpy as np
import os 
import openai
import csv
import requests
import json
import re
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [2]:
def get_bearer_token():
    
    """ Obtain token to access endpoint"""
    
    client_id = '96c2a636-5226-4de9-874f-01448d3f5672'
    scope = 'https://cognitiveservices.azure.com'
    secret = ''
    tenant_id = 'fb007914-6020-4374-977e-21bac5f3f4c8'

    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {'Content-Type':'application/x-www-form-urlencoded',
              'Cookie': 'fpc=Anq1T2lObB1IgSf3CzTX1zR0bXNTAQAAAG-rndwOAAAA; stsservicecookie=estsfd; x-ms-gateway-slice=estsfd'}

    payload = f'client_id={client_id}&scope={scope}/.default&client_secret={secret}&grant_type=client_credentials'
    response = requests.post(url=url,data=payload,headers = headers,timeout=60)
    resp_data = response.json()
    return resp_data.get('access_token',"")

In [53]:
azure_url = "https://genai-cus-tdz.paypalcorp.com/pmts-smb-ai"
openai.api_base = azure_url
openai.api_key = get_bearer_token()
openai.api_type = "azure_ad"
openai.api_version = "2024-12-01-preview" #"2024-05-01-preview"
openai.verify_ssl_certs = False
# openai.pp_customization = True
deployment_name = "gpt-4o" #"gpt-4_0613"

In [4]:
def get_completion_and_token_count(prompt, engine=deployment_name):

    """Returns prompt response and the number of tokens used"""
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=0
    )
    content =  response.choices[0].message["content"]
    token_dict = {
    'prompt_tokens':response['usage']['prompt_tokens'],
    'completion_tokens':response['usage']['completion_tokens'],
    'total_tokens':response['usage']['total_tokens'],
    }
    return content #, token_dict

In [5]:
base_df = pd.read_csv("base_tasks_train_data.csv")

In [10]:
base_qs = list(base_df['user_query'].unique())

In [57]:
all_questions = {}

In [30]:
new_df_list = []
new_list_errs = []

In [99]:
for q in base_qs[5:]: #run from 5 next
    print("-"*50)
    print(q)
    # for 1st question, obtained 49 similar questions
    # obtain 30 similar questions
    query = """Your task is to give sample requests that users might ask a Venmo assistant. 
    Could you please provide 30 questions similar to the one given below:
    ```{}```
    Make sure to include different English dialects like Boston, New York, etc. Ensure it is still polite without too much slang.
    Make the questions similar but varying slightly. Ensure the same format is followed.
    Return only the questions. No additional text.
    Separate each example by a new line. No numbering.
    """.format(q)
    try:
        resp = get_completion_and_token_count(query)
    except:
        openai.api_key = get_bearer_token()
        resp = get_completion_and_token_count(query)
        
    list_resp = [i.strip() for i in resp.split("\n") if i.strip()!='']
    all_questions[q] = list_resp
    print(len(list_resp),len(set(list_resp)))

    sub_df = base_df[base_df['user_query']==q]
    for i in list_resp:
        print(i)
        user_instruction = i
        for row_num in range(len(sub_df)):
            prompt_gen_data = """You are a helpful assistant tasked with generating synthetic data for training a conversational assistant. The goal is to help carry out users requests. 
            The following is a sample data point that is available to us for a specific user query:
            
            {{ "prompt":{} , "completion":{} }}
            
            Create a similar json containing the "prompt" and "completions" keys for this user query - {}
            Use the same template as the example provided above, only change the relevant details. Your response should be very similar to the example.
            Do not confuse the text between the angular brackets as instruction, it is just a template. While returning this template as part of the "prompt" key, make sure to include the expect output format as mentioned in the example.
            It should closely resemble the given data point. However, ensure to include the right details from the user query.
            Return only the json string. No additional text.
            Simply copy what is given in the example but change the details with respect to what is mentioned in user's instructions.
            Do not add your points for thought or action.
            Return your response in the following format:
            {{ "prompt" : "<your response here>", 
            "completion" : "<your response here>" }}
            """.format(sub_df.iloc[row_num]["prompt"],sub_df.iloc[row_num]["completion"],i)
            
            try:
                samp_resp = get_completion_and_token_count(prompt_gen_data)
            except:
                openai.api_key = get_bearer_token()
                samp_resp = get_completion_and_token_count(prompt_gen_data)
    
            if samp_resp[-1]=="`" and samp_resp[0]=="`":
                json_str = samp_resp[7:-3].strip()
            else:
                json_str = samp_resp.strip()
            
            try:
                json_str = json_str.replace("`","\'").replace("\n"," ")
                new_row = json.loads(json_str)
                try:
                    new_df_list.append({"user_query":user_instruction, "prompt":new_row["prompt"], "completion":new_row["completion"]})
                except:
                    print(i,row_num)
                    new_list_errs.append({"user_query":user_instruction,"response":json_str})
                    
            except json.JSONDecodeError as e:
                print(i,row_num)
                new_list_errs.append({"user_query":user_instruction,"response":json_str})
                

--------------------------------------------------
I paid for our last month's electricity bill. Its amount is supposed to be shared equally among my roommates and me. Make venmo requests to my roommates, with a description note, 'For electricity bill.'. The bill receipt is in my file system.


/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


30 30
I covered the tab for dinner last night. Split the cost evenly among my friends and send Venmo requests with the note, 'Dinner at Joe's.' The receipt is saved in my files.


/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

I covered the tab for dinner last night. Split the cost evenly among my friends and send Venmo requests with the note, 'Dinner at Joe's.' The receipt is saved in my files. 8


...

/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

We had lunch at The Quartz Café yesterday, and my friend covered the bill. Everyone Venmoed them their share, but my $19 payment isn’t on the feed. How much did they pay for everyone, including me?


/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

I went out for coffee with my friends at The Daily Grind yesterday, and my buddy paid for everyone. Everyone Venmoed them their share, but my $11 payment isn’t showing on the feed. Can you tell me how much they spent for the group, including me?


/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/asankaranarayana/Documents/rg_data/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-cus-tdz.paypalcorp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

In [100]:
len(new_df_list)

1595

In [101]:
temp_df = pd.DataFrame(new_df_list)
len(temp_df)

1595

In [102]:
temp_df.to_csv("simulated_tasks_train.csv",index=False)

In [103]:
len(new_list_errs)

139

In [104]:
with open("generated_err_samples.txt", "w") as file:
        for item in new_list_errs:
            file.write(str(item) + "\n")

In [107]:
content = []
with open("generated_err_samples.txt", "r") as file:
    for line in file:
        content.append(line.strip())

In [109]:
len(content)

139

In [119]:
content[0]

'{\'user_query\': \'Send $250 on Venmo to the person with this username @emma_boston, make it public.\', \'response\': \'{   "prompt": "User Instruction :  \\\'\\\'\\\'Send $250 on Venmo to the person with this username @emma_boston, make it public.\\\'\\\'\\\' The following list contains the ordered set of steps already taken, the last element of the list is the current state of the task :  \\\'\\\'\\\'[\\\\"First, let\\\'s check the available APIs for the Venmo app to see how we can send money.\\\\", \\\\"The \\\'create_transaction\\\' API seems to be the one we need to send money. Let\\\'s check its specifications.\\\\", \\\'we can proceed to send $250 to the person with the username @emma_boston. First, we need to verify the username using the \\\\\\\'venmo\\\\\\\' app.\\\']\\\'\\\'\\\' Current Observation :  \\\'\\\'\\\'{\\\'app_name\\\': \\\'venmo\\\', \\\'api_name\\\': \\\'search_users\\\', \\\'path\\\': \\\'/users\\\', \\\'method\\\': \\\'GET\\\', \\\'description\\\': \\\'Searc

In [117]:
j_str = content[0]
j_str = j_str.replace("\\\","\\")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2128941808.py, line 1)

In [120]:
json_str

'{      "prompt" : "User Instruction :  \'\'\'I went out for coffee with my friends at The Daily Grind yesterday, and my buddy paid for everyone. Everyone Venmoed them their share, but my $11 payment isn’t showing on the feed. Can you tell me how much they spent for the group, including me?\'\'\' The following list contains the ordered set of steps already taken, the last element of the list is the current state of the task :  \'\'\'[\\"First, let\'s check the available APIs for the Venmo app to see how we can retrieve the transactions.\\", \\"The \'show_social_feed\' API seems to be the one we need to retrieve the transactions. Let\'s check its specifications.\\", \'Now, I retrieve the social feed and look for transactions related to the coffee at The Daily Grind. I will filter the transactions to find those related to \\"The Daily Grind\\" and sum the amounts to find out how much your buddy paid for the group, including you.\']\'\'\' Current Observation :  \'\'\'55.0\'\'\'  Based on 

In [ ]:
'{   "prompt": "User Instruction :  \'\'\'Send $250 on Venmo to the person with this username @emma_boston, make it public.\'\'\' 
The following list contains the ordered set of steps already taken, the last element of the list is the current state of the task :  
\'\'\'[\\"First, let\'s check the available APIs for the Venmo app to see how we can send money.\\", 
\\"The \'create_transaction\' API seems to be the one we need to send money. Let\'s check its specifications.\\",
\'we can proceed to send $250 to the person with the username @emma_boston. First, we need to verify the username using the \\\\\\\'venmo\\\\\\\' app.\\\']\\\'\\\'\\\' Current Observation :  \\\'\\\'\\\'{\\\'app_name\\\': \\\'venmo\\\', \\\'api_name\\\': \\\'search_users\\\', \\\'path\\\': \\\'/users\\\', \\\'method\\\': \\\'GET\\\', \\\'description\\\': \\\'Search for users on Venmo by username or other details.\\\', \\\'parameters\\\': [{\\\'name\\\': \\\'query\\\', \\\'type\\\': \\\'string\\\', \\\'required\\\': True, \\\'description\\\': \\\'Search query for the user.\\\', \\\'default\\\': \\\'\\\', \\\'constraints\\\': []}, {\\\'name\\\': \\\'page_index\\\', \\\'type\\\': \\\'integer\\\', \\\'required\\\': False, \\\'description\\\': \\\'The index of the page to return.\\\', \\\'default\\\': 0, \\\'constraints\\\': [\\\'value >= 0.0\\\']}, {\\\'name\\\': \\\'page_limit\\\', \\\'type\\\': \\\'integer\\\', \\\'required\\\': False, \\\'description\\\': \\\'The maximum number of results to return per page.\\\', \\\'default\\\': 5, \\\'constraints\\\': [\\\'value >= 1.0, <= 20.0\\\']}], \\\'response_schemas\\\': {\\\'success\\\': [{\\\'user_id\\\': 1, \\\'username\\\': \\\'string\\\', \\\'first_name\\\': \\\'string\\\', \\\'last_name\\\': \\\'string\\\', \\\'email\\\': \\\'string\\\', \\\'phone_number\\\': \\\'string\\\', \\\'created_at\\\': \\\'2019-01-01T00:00:00\\\'}], \\\'failure\\\': {\\\'message\\\': \\\'string\\\'}}}\\\'\\\'\\\'  Based on User instruction, history of steps taken and current observation, decide your next step. Respond with the action to take and the thought behind it.  Respond in the following json format: {\\\\"thought\\\\" : \\\\"...\\\\", \\\\"action\\\\" : \\\\"...\\\\"}  , \\\\"completion\\\\":{\\\'thought\\\': \\\\"Let\\\'s search for the user with the username @emma_boston using the \\\'venmo\\\' app.\\\\", \\\'action\\\': \\\\"user_search_result = apis.venmo.search_users(query=\\\'@emma_boston\\\') \\\\\\\\n print(user_search_result)\\\\"}",   "completion": {     "thought": "Let\\\'s search for the user with the username @emma_boston using the \\\'venmo\\\' app.",     "action": "user_search_result = apis.venmo.search_users(query=\\\'@emma_boston\\\') \\\\n print(user_search_result)"   } }\'}'

In [140]:
sample_str = content[0].replace("\\\'","\'").replace('\\\\"','\\"').replace('}"','"}')

In [141]:
sample_str

'{\'user_query\': \'Send $250 on Venmo to the person with this username @emma_boston, make it public.\', \'response\': \'{   "prompt": "User Instruction :  \'\'\'Send $250 on Venmo to the person with this username @emma_boston, make it public.\'\'\' The following list contains the ordered set of steps already taken, the last element of the list is the current state of the task :  \'\'\'[\\"First, let\'s check the available APIs for the Venmo app to see how we can send money.\\", \\"The \'create_transaction\' API seems to be the one we need to send money. Let\'s check its specifications.\\", \'we can proceed to send $250 to the person with the username @emma_boston. First, we need to verify the username using the \\\\\'venmo\\\\\' app.\']\'\'\' Current Observation :  \'\'\'{\'app_name\': \'venmo\', \'api_name\': \'search_users\', \'path\': \'/users\', \'method\': \'GET\', \'description\': \'Search for users on Venmo by username or other details.\', \'parameters\': [{\'name\': \'query\',

In [142]:
pattern = r'\\\'response\\\'\:\\\'(.*?)'
match = re.search(pattern, sample_str, re.DOTALL)